In [1]:
from loaders.Loaders import CSVLoader
from compoundFeaturization.rdkitFingerprints import MorganFingerprint
from featureSelection.baseFeatureSelector import LowVarianceFS
from splitters.splitters import SingletaskStratifiedSplitter
from models.sklearnModels import SklearnModel
from metrics.Metrics import Metric
from metrics.metricsFunctions import r2_score, roc_auc_score, precision_score, accuracy_score, confusion_matrix, classification_report, f1_score
from parameterOptimization.HyperparameterOpt import HyperparamOpt_CV, HyperparamOpt_Valid

from sklearn.ensemble import RandomForestClassifier

Using TensorFlow backend.


In [3]:
#Load Dataset
dataset = CSVLoader(dataset_path='data/preprocessed_dataset_wfoodb.csv', 
                    mols_field='Smiles', 
                    labels_fields='Class', 
                    id_field='ID')#, shard_size=4000)
dataset = dataset.create_dataset()
dataset.get_shape()

Mols_shape:  23290
Features_shape:  X not defined!
Labels_shape:  (23290,)


In [4]:
#Featurization
dataset = MorganFingerprint().featurize(dataset)
dataset.get_shape()

Featurizing datapoint 0
Featurizing datapoint 1000
Featurizing datapoint 2000
Featurizing datapoint 3000
Featurizing datapoint 4000
Featurizing datapoint 5000
Featurizing datapoint 6000


RDKit ERROR: [09:45:07] Explicit valence for atom # 1 Cl, 4, is greater than permitted


error in smile: O=[Cl]=O
Featurizing datapoint 7000


RDKit ERROR: [09:45:13] Explicit valence for atom # 3 B, 4, is greater than permitted
RDKit ERROR: [09:45:13] Explicit valence for atom # 1 Cl, 9, is greater than permitted


error in smile: OB1O[B]2(O)OB(O)O[B](O)(O1)O2
error in smile: O=[Cl-](=O)(=O)=O
Featurizing datapoint 8000
Featurizing datapoint 9000
Featurizing datapoint 10000
Featurizing datapoint 11000
Featurizing datapoint 12000
Featurizing datapoint 13000
Featurizing datapoint 14000


RDKit ERROR: [09:45:40] Explicit valence for atom # 0 P, 11, is greater than permitted


error in smile: [P](OCC=C(C)C)(OCC=C(C)C)(=O)(OP(OCC=C(C)C)(OCC=C(C)C)=O)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)CC=C(C)C
Featurizing datapoint 15000
Featurizing datapoint 16000
Featurizing datapoint 17000
Featurizing datapoint 18000
Featurizing datapoint 19000
Featurizing datapoint 20000
Featurizing datapoint 21000
Featurizing datapoint 22000
Featurizing datapoint 23000
Elements with indexes:  [6257, 7708, 7709, 14244]  were removed due to the presence of NAs!
The elements in question are:  ['O=[Cl]=O' 'OB1O[B]2(O)OB(O)O[B](O)(O1)O2' 'O=[Cl-](=O)(=O)=O'
 '[P](OCC=C(C)C)(OCC=C(C)C)(=O)(OP(OCC=C(C)C)(OCC=C(C)C)=O)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)CC=C(C)C']
Mols_shape:  23286
Features_shape:  (23286, 2048)
Labels_shape:  (23286,)


In [5]:
#Feature Selection
dataset = LowVarianceFS(0.15).featureSelection(dataset)
dataset.get_shape()

Mols_shape:  23286
Features_shape:  (23286, 44)
Labels_shape:  (23286,)


In [6]:
#Data Split
splitter = SingletaskStratifiedSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset=dataset, frac_train=0.6, 
                                                                             frac_valid=0.2, frac_test=0.2)

(13968, 44)
(4656, 44)
(4656, 44)


In [7]:
#Scikit-Learn Random Forest
rf = RandomForestClassifier()
model = SklearnModel(model=rf)

In [11]:
#cross validation
model.cross_validate(dataset, Metric(roc_auc_score), folds=3)

Computing Stratified K-fold split
(15524, 44)
(7762, 44)
(15524, 44)
(7762, 44)
(15524, 44)
(7762, 44)

Split 1 :
Train Score: 
roc_auc_score: 
 0.8163949821607831
Test Score: 
roc_auc_score: 
 0.6438296168458691

Split 2 :
Train Score: 
roc_auc_score: 
 0.8150548316679306
Test Score: 
roc_auc_score: 
 0.6375165282613524

Split 3 :
Train Score: 
roc_auc_score: 
 0.7997442808924287
Test Score: 
roc_auc_score: 
 0.652829387457756


(SklearnModel(model=RandomForestClassifier()),
 0.7997442808924287,
 0.652829387457756,
 [0.8163949821607831, 0.8150548316679306, 0.7997442808924287],
 [0.6438296168458691, 0.6375165282613524, 0.652829387457756],
 0.8103980315737142,
 0.6447251775216591)

In [9]:
# model training
model.fit(train_dataset)

RandomForestClassifier()

In [10]:
metrics = [Metric(roc_auc_score), Metric(precision_score), Metric(accuracy_score), Metric(confusion_matrix), 
           Metric(classification_report)]

# evaluate the model
print('Training Dataset: ')
train_score = model.evaluate(train_dataset, metrics)

print('\n\nValidation Dataset: ')
valid_score = model.evaluate(valid_dataset, metrics)

print('\n\nTest Dataset: ')
test_score = model.evaluate(test_dataset, metrics)


Training Dataset: 
roc_auc_score: 
 0.8043855865293515
precision_score: 
 0.8977695167286245
accuracy_score: 
 0.9742268041237113
confusion_matrix: 
 [[13125    55]
 [  305   483]]
classification_report: 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99     13180
           1       0.90      0.61      0.73       788

    accuracy                           0.97     13968
   macro avg       0.94      0.80      0.86     13968
weighted avg       0.97      0.97      0.97     13968



Validation Dataset: 
roc_auc_score: 
 0.6419937075674781
precision_score: 
 0.541095890410959
accuracy_score: 
 0.9458762886597938
confusion_matrix: 
 [[4325   67]
 [ 185   79]]
classification_report: 
               precision    recall  f1-score   support

           0       0.96      0.98      0.97      4392
           1       0.54      0.30      0.39       264

    accuracy                           0.95      4656
   macro avg       0.75      0.64      0.68 

In [10]:
#Build a model function for hyperparameter optimization
def rf_model_builder(n_estimators=10, max_features='auto', class_weight={0: 1., 1: 1.}):
    rf_model = RandomForestClassifier(n_estimators=n_estimators, max_features=max_features, 
                                      class_weight=class_weight)
    return rf_model

params_dict_rf = {"n_estimators": [10, 100],
                  "max_features": ["auto", "sqrt", "log2", None],
                  "class_weight": [{0: 1., 1: 1.}, {0: 1., 1: 5}, {0: 1., 1: 10}]
                  }


In [11]:
#Hyperparameter Optimization
optimizer = HyperparamOpt_Valid(rf_model_builder)

best_rf, best_hyperparams, all_results = optimizer.hyperparam_search(params_dict_rf, 
                                                                     train_dataset, 
                                                                     valid_dataset, 
                                                                     Metric(accuracy_score))

print('#################')
print(best_hyperparams)
print(best_rf)

#Evaluate model
best_rf.evaluate(test_dataset, metrics)

MODE:  classification
Fitting 15 random models from a space of 24 possible models.
Fitting model 1/15
hyperparameters: {'n_estimators': 10, 'max_features': 'sqrt', 'class_weight': {0: 1.0, 1: 1.0}}
expected str, bytes or os.PathLike object, not NoneType
accuracy_score: 
 0.9450171821305842
Model 1/15, Metric accuracy_score, Validation set 1: 0.945017
	best_validation_score so far: 0.945017
Fitting model 2/15
hyperparameters: {'n_estimators': 10, 'max_features': 'sqrt', 'class_weight': {0: 1.0, 1: 5}}
expected str, bytes or os.PathLike object, not NoneType
accuracy_score: 
 0.9372852233676976
Model 2/15, Metric accuracy_score, Validation set 2: 0.937285
	best_validation_score so far: 0.945017
Fitting model 3/15
hyperparameters: {'n_estimators': 10, 'max_features': 'sqrt', 'class_weight': {0: 1.0, 1: 10}}
expected str, bytes or os.PathLike object, not NoneType
accuracy_score: 
 0.9317010309278351
Model 3/15, Metric accuracy_score, Validation set 3: 0.931701
	best_validation_score so far:

{'roc_auc_score': 0.6477324364115397,
 'precision_score': 0.5955882352941176,
 'accuracy_score': 0.9490979381443299,
 'confusion_matrix': 1164.0,
 'classification_report': None}

In [12]:
#Hyperparameter Optimization with CV
optimizer = HyperparamOpt_CV(rf_model_builder)

best_rf, best_hyperparams, all_results = optimizer.hyperparam_search('sklearn',
                                                                     params_dict_rf, 
                                                                     train_dataset,  
                                                                     'accuracy', 
                                                                     cv=3,
                                                                     n_iter_search=10)

print('#################')
print(best_hyperparams)
print(best_rf)

MODEL TYPE:  sklearn
Fitting 10 random models from a space of 24 possible models.
RandomForestClassifier(class_weight={0: 1.0, 1: 1.0}, n_estimators=10)

 
 Best accuracy: 0.936068 using {'n_estimators': 10, 'max_features': 'auto', 'class_weight': {0: 1.0, 1: 1.0}}

 accuracy: 0.921893 (0.005206) with: {'n_estimators': 100, 'max_features': 'sqrt', 'class_weight': {0: 1.0, 1: 10}} 


 accuracy: 0.926904 (0.005209) with: {'n_estimators': 100, 'max_features': 'log2', 'class_weight': {0: 1.0, 1: 5}} 


 accuracy: 0.924399 (0.005333) with: {'n_estimators': 10, 'max_features': 'log2', 'class_weight': {0: 1.0, 1: 5}} 


 accuracy: 0.925687 (0.004650) with: {'n_estimators': 10, 'max_features': 'auto', 'class_weight': {0: 1.0, 1: 5}} 


 accuracy: 0.924255 (0.005035) with: {'n_estimators': 10, 'max_features': None, 'class_weight': {0: 1.0, 1: 5}} 


 accuracy: 0.927549 (0.005090) with: {'n_estimators': 100, 'max_features': 'sqrt', 'class_weight': {0: 1.0, 1: 5}} 


 accuracy: 0.922322 (0.005010

In [13]:
#Evaluate model
best_rf.evaluate(test_dataset, metrics)

roc_auc_score: 
 0.6562938445972205
precision_score: 
 0.5209580838323353
accuracy_score: 
 0.9450171821305842
confusion_matrix: 
 [[4313   80]
 [ 176   87]]
classification_report: 
               precision    recall  f1-score   support

           0       0.96      0.98      0.97      4393
           1       0.52      0.33      0.40       263

    accuracy                           0.95      4656
   macro avg       0.74      0.66      0.69      4656
weighted avg       0.94      0.95      0.94      4656



{'roc_auc_score': 0.6562938445972205,
 'precision_score': 0.5209580838323353,
 'accuracy_score': 0.9450171821305842,
 'confusion_matrix': 1164.0,
 'classification_report': None}